Leave one out validation

In [ ]:
import numpy
import tensorflow
from tensorflow import keras
import matplotlib.pyplot as plt
from Bio import SeqIO

In [ ]:
def from_files_to_vectors(fasta_path, positive=True):
    if positive:
        proteins = list(SeqIO.parse(fasta_path+'pos_25.fasta', "fasta"))
    else:
        proteins = list(SeqIO.parse(fasta_path+'neg_25.fasta', "fasta"))
    extension = ".out"
    files = ["aac", "dpc", "ctdc", "ctdt", "ctdd"]
    if positive:
        names = "_pos"
    else:
        names = "_neg"
    for i in range(len(files)):
        files[i] += names
    datasets = [[] for el in files]
    for i in range(len(files)):
        with open(fasta_path+files[i]+extension) as f:
            lines = f.readlines()[1:]
            check_prot = 0
            for line in lines:
                information = line.split('\t')
                if not information[0] == proteins[check_prot].id:
                    print("Error in protein order! Return")
                    return datasets
                datasets[i].append(numpy.array([float(el) for el in information[1:]]))
                check_prot += 1
        datasets[i] = numpy.array(datasets[i])
    return datasets

In [ ]:
pos_datasets = from_files_to_vectors("./Adhesin_data/25_similarity/pos/", positive=True)
neg_datasets = from_files_to_vectors("./Adhesin_data/25_similarity/neg/", positive=False)

y_pos = numpy.ones(pos_datasets[0].shape[0])
y_neg = numpy.zeros(neg_datasets[0].shape[0])

In [ ]:
# attach datasets in order to obtain a matrix of (n, 20+400+39+39+195) features

# keep in mind the number of virulent factors and the number of not virulent factors
rows = 0
n_pos = y_pos.shape[0]
n_neg = y_neg.shape[0]
rows = n_pos + n_neg
print('pos:', n_pos)
print('neg:', n_neg)

# feature vectors dimensions
columns = 0
for i in range(len(pos_datasets)):
    tmp_dim = pos_datasets[i].shape[1]
    print(i+1, '-th feature dim:', tmp_dim)
    columns += tmp_dim

# data matrix to process
X = numpy.zeros((rows, columns))
print('Data matrix dimension:', X.shape)
for i in range(n_pos):
    X[i] = numpy.concatenate([pos_datasets[j][i] for j in range(5)])
for i in range(n_neg):
    X[n_pos+i] = numpy.concatenate([neg_datasets[j][i] for j in range(5)])

In [ ]:
# permutation
numpy.random.seed(990)

y = numpy.concatenate((y_pos, y_neg), axis=0)
c = numpy.random.permutation(numpy.arange(y.shape[0]))
y = y[c]
X = X[c] 

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
# remember: standardization or scaling AFTER train-test split, otherwise there's data leakage!!!
X_train = X[:int(X.shape[0]*.5)]
X_val = X[int(X.shape[0]*.5):int(X.shape[0]*.75)]
X_test = X[int(X.shape[0]*.75):]

y_train = y[:int(y.shape[0]*.5)]
y_val = y[int(y.shape[0]*.5):int(y.shape[0]*.75)]
y_test = y[int(y.shape[0]*.75):]

print('Training data shape:', X_train.shape, y_train.shape)
print('Validation data shape:', X_val.shape, y_val.shape)
print('Test data shape:', X_test.shape, y_test.shape)

print('\nTraining virulent factors and not-virulent factors:', int(sum(y_train)), int(y_train.shape[0]-sum(y_train)))
print('Validation virulent factors and not-virulent factors:', int(sum(y_val)), int(y_val.shape[0]-sum(y_val)))
print('Test virulent factors and not-virulent factors:', int(sum(y_test)), int(y_test.shape[0]-sum(y_test)))

In [ ]:
# PCA

stdX = numpy.zeros(X_train.shape)
stdX_val = numpy.zeros(X_val.shape)
stdX_test = numpy.zeros(X_test.shape)

means = numpy.zeros(X_train.shape[1])
std_devs = numpy.zeros(X_train.shape[1])

for j in range(X_train.shape[1]):
    column = X_train[:,j]
    means[j] = numpy.mean(column)
    std_devs[j] = numpy.std(column)
    stdX[:,j] = (column - means[j]) / std_devs[j]
    stdX_val[:,j] = (X_val[:,j] - means[j]) / std_devs[j]
    stdX_test[:,j] = (X_test[:,j] - means[j]) / std_devs[j]

numpy.save('means', means)
numpy.save('std_devs', std_devs)

covariance_matrix = numpy.cov(stdX.T)

eigen_values, eigen_vectors = numpy.linalg.eig(covariance_matrix)
eigen_values = numpy.real(eigen_values)
eigen_vectors = numpy.real(eigen_vectors)

variance_explained = []
for i in eigen_values:
    variance_explained.append((i/sum(eigen_values))*100)

cumulative_variance_explained = numpy.cumsum(variance_explained)

In [ ]:
plt.title("Explained variance vs Number of components")

plt.xlabel("Number of components")
plt.ylabel("Cumulative explained variance")
plt.grid(color='gray', linewidth=.4)

plt.plot(range(len(cumulative_variance_explained)), cumulative_variance_explained)

plt.show()
#plt.savefig('Explained variance.png')

In [ ]:
K = 400 # more or less like adhesin
print('Principal components:', K)
print('Discarded components:', columns-K)

In [ ]:
plt.title("Explained variance vs first " + str(K) + " components")

plt.xlabel("Number of components")
plt.ylabel("Cumulative explained variance")
plt.grid(color='gray', linewidth=.4)

plt.plot(range(K), cumulative_variance_explained[:K])

plt.show()
#plt.savefig('Explained variance.png')

In [ ]:
# Project using first K components

projection_matrix = numpy.real((eigen_vectors.T[:][:K]).T)
print(projection_matrix.shape)

numpy.save('projection_matrix', projection_matrix)

In [ ]:
# project matrices
X_train = stdX.dot(projection_matrix)
X_val = stdX_val.dot(projection_matrix)
X_test = stdX_test.dot(projection_matrix)

In [ ]:
from tensorflow.keras import regularizers

class neural_network:
    def __init__(self):
        input = tensorflow.keras.Input(shape=(K,))

        dense = tensorflow.keras.layers.Dense(units=10, activation='sigmoid')(input) # it shows that a linear activation is sufficiet to get the same accuracy of the sigmoid (so it's like a matrix multiplication with a non-linearity at the end...)
        #batch = tensorflow.keras.layers.BatchNormalization()(dense)
        #activation = tensorflow.keras.activations.sigmoid(batch)
        #drop = tensorflow.keras.layers.Dropout(.1)(activation)

        output = tensorflow.keras.layers.Dense(1, activation='sigmoid')(dense)

        model = tensorflow.keras.models.Model(inputs=input, outputs=output)
        model.compile(optimizer=tensorflow.keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy', 'mse', 'recall'])
        self.model = model

In [ ]:
nn = neural_network()

In [ ]:
history = nn.model.fit(
    x=X_train, 
    y=y_train,
    batch_size=64,
    epochs=1000,
    verbose=1,
    validation_data=(X_val, y_val),
    shuffle=True,
    callbacks=[tensorflow.keras.callbacks.EarlyStopping(
    restore_best_weights=True,
    patience=20
        )]
)

In [ ]:
plt.title('Accuracy during training')
plt.plot(range(len(history.history['loss'])), history.history['accuracy'], label='training accuracy', color='green')
plt.plot(range(len(history.history['loss'])), history.history['val_accuracy'], label='validation accuracy', color='red')
plt.grid(color='gray', linewidth=.4)

plt.legend(loc="lower right")
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.xlim(0, len(history.history['loss'])+1)
plt.ylim(.45, 1.)
plt.savefig('acc.png')

In [ ]:
plt.title('Loss during training')
plt.plot(range(len(history.history['loss'])), history.history['loss'], label='training loss', color='green')
plt.plot(range(len(history.history['loss'])), history.history['val_loss'], label='validation loss', color='red')
plt.grid(color='gray', linewidth=.4)

plt.legend(loc="upper right")
plt.xlabel("epochs")
plt.ylabel("loss")
plt.savefig('loss.png')

In [ ]:
nn.model.evaluate(x=X_test, y=y_test)

In [ ]:
# Evaluate prediction distributions
predictions = nn.model.predict(X_test)
ground_truth = y_test

In [ ]:
# get true and false negative and positives
results = numpy.c_[predictions, ground_truth]
TP = results[numpy.where(((results[:,1] == 1) & (results[:,0] > .50)))]
TN = results[numpy.where((results[:,1] == 0) & (results[:,0] < .50))]
FP = results[numpy.where(((results[:,1] == 0) & (results[:,0] > .50)))]
FN = results[numpy.where((results[:,1] == 1) & (results[:,0] < .50))]

In [ ]:
plt.figure(figsize=[9,5])
plt.hist(numpy.append(TP[:,0], TN[:,0]), bins=20, alpha=0.5, label = 'True positives and True negatives')
plt.hist(numpy.append(FP[:,0], FN[:,0]), bins=20, alpha=0.5, label = 'False positives and False negatives')
plt.xticks(numpy.arange(0, 1.05, step=0.05))
plt.legend()
plt.title('Virulent predictions on test set')
plt.xlabel('Predicted value')
plt.show()

In [ ]:
accuracy = (len(TP) + len(TN)) / len(results)
sensitivity = len(TP) / (len(TP) + len(FN))
precision = len(TP) / (len(TP) + len(FP))
f1_score = 2 * (precision * sensitivity) / (precision + sensitivity)
FPR = len(FP) / (len(FP) + len(TN))
FNR = len(FN)/ (len(TP)+len(FN))
TNR = len(TN)/(len(TN)+len(FP))
FDR = len(FP)/(len(FP)+len(TP))
TPR = len(TP)/(len(TP)+len(FN))
NPV = len(TN)/(len(TN)+len(FN))

print("Accuracy:", accuracy)
print("Sensitivity (Recall):", sensitivity)
print("Precision:", precision)
print("F1-score:", f1_score)
print("False Positive Rate (FPR):", FPR)
print("False Negative Rate (FNR):", FNR)
print("True Negative Rate (TNR):", TNR)
print("False Discovery Rate (FDR):", FDR)
print("True Positive Rate (FDR):", TPR)
print("Negative Predictive Value (NPV):", NPV)

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns

y_pred_class = predictions >= 0.51
cm = confusion_matrix(ground_truth, y_pred_class)
TN, FP, FN, TP = cm.ravel()

In [ ]:
cm.ravel()

In [ ]:
confusion_matrix = metrics.confusion_matrix(ground_truth, y_pred_class)

In [ ]:
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = ["Not Adhesin", "Adhesin"])

In [ ]:
cm_display.plot(cmap='Blues')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Supponendo che cm_display.plot(cmap='Blues') sia già stato eseguito
cm_display.plot(cmap='Blues')

# Salva il grafico in un file PNG a 300 DPI
plt.savefig('conf_matrix_adh.png', dpi=330)

# Mostra il grafico
plt.show()